In [4]:
import feedparser
from bs4 import BeautifulSoup
import requests

In [5]:
# Define the default name and feed of the news outlet
NEWS_OUTLET = "Telegraph"
NEWS_FEED = "https://www.telegraph.co.uk/rss.xml"
NEWS_LANGUAGE = "en-UK"

In [19]:
# Read the RSS feed and retrieve URL and article metadata
def get_rss_feed():
    article_list = []
    newsFeed = feedparser.parse(NEWS_FEED)

    for rss_article in newsFeed.entries:

        # Collection to hold the article specific metadata
        article_props = {}
        article_props['url'] = rss_article.link
        article_props['title'] = rss_article.title
        #article_props['lead'] = rss_article.summary
        #article_props['author'] = rss_article.authors
        #article_props['primaryCategory'] = rss_article.tags
        article_props['date_published'] = rss_article.published
        article_props['image'] = rss_article.links

        article_list.append(article_props)
        
    return article_list

In [7]:
article_list = []
newsFeed = feedparser.parse(NEWS_FEED)

for rss_article in newsFeed.entries:
    print(rss_article)

{'title': 'The brains behind Beyoncé: who’s who in her inner circle', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.telegraph.co.uk/rss.xml', 'value': 'The brains behind Beyoncé: who’s who in her inner circle'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.telegraph.co.uk/music/artists/beyonce-renaissance-tour-team-whos-who/'}, {'length': '99', 'type': 'image/jpeg', 'href': 'https://www.telegraph.co.uk/content/dam/music/2023/05/15/TELEMMGLPICT000335162884_trans_NvBQzQNjv4BqkYZvbd7oZeZE85AXAfeOBYA-LKtAk7yxsdUDTllG-JI.jpeg', 'rel': 'enclosure'}], 'link': 'https://www.telegraph.co.uk/music/artists/beyonce-renaissance-tour-team-whos-who/', 'id': '85f38ca4-d0ca-3fde-9499-62c5055a9e58', 'guidislink': False, 'published': 'Mon, 15 May 2023 16:54:30 GMT', 'published_parsed': time.struct_time(tm_year=2023, tm_mon=5, tm_mday=15, tm_hour=16, tm_min=54, tm_sec=30, tm_wday=0, tm_yday=135, tm_isdst=0), 'authors': [{'name': 'Ed Power'}], 'aut

In [8]:
from bson.objectid import ObjectId
from datetime import datetime

def generate_id():
    return str(ObjectId())

def create_article(
    *,
    url,
    primary_category,
    sub_categories = [],
    title,
    lead,
    author = None,
    date_published,
    date_updated = None,
    language,
    outlet,
    image = None,
    body,
):
    return {
        "_id": generate_id(), # Generate custom ID because the backend uses strings instead of ObjectId()s
        "url": url,
        "primaryCategory": primary_category,
        "subCategories": sub_categories,
        "title": title,
        "lead": lead,
        "author": author,
        "datePublished": date_published,
        "dateScraped": datetime.now(),
        "dateUpdated": date_updated,
        "language": language,
        "outlet": outlet,
        "image": image,
        "body": body,
    }

In [131]:
def scrape_article(article):
    
    response = requests.get(article['url'])
    soup = BeautifulSoup(response.content, 'html.parser')
    
    body = soup.find_all('div', {'class': 'component article-body-text'})
    primary_category = soup.find('li', {'class': 'e-navigation-primary-item e-navigation-primary-item--current'}).get_text()
    sub_categories = 
    try:
        soup.find('li', {'class': 'e-navigation-secondary-item e-navigation-secondary-item--current e-navigation-secondary-item--has-children accordion accordion--sm accordion--is-collapsed e-navigation-secondary-item--first '})
        except:
            "no subcategory"
    src = soup.find('img', {'class': 'article-body-image__source'}).get("src")
    image = 'https://www.telegraph.co.uk' + src

    author = soup.find('span', {'class': 'e-byline__author'}).get("content")
    teaser= try:
        soup.find('p', {'class': 'e-standfirst u-heading-4'}).get_text()
        except:
            "no teaser"
    date_updated = "test"
    
    document = create_article(
        url=article['url'],
        primary_category= primary_category,
        sub_categories=sub_categories,
        title=article['title'],
        lead=teaser,
        author=author,
        date_published="test",
        date_updated=date_updated, 
        language=NEWS_LANGUAGE,
        outlet=NEWS_OUTLET,
        image=image,
        body=body
    )

    print(document)

In [106]:
rss_results = get_rss_feed()
for article in rss_results:
    print(article)

{'url': 'https://www.telegraph.co.uk/news/2023/05/15/mountain-rescue-dog-scafell-pike-lake-district/', 'title': 'Mountain rescue called over dog that refused to walk back down Scafell Pike', 'date_published': 'Mon, 15 May 2023 17:40:12 GMT', 'image': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.telegraph.co.uk/news/2023/05/15/mountain-rescue-dog-scafell-pike-lake-district/'}, {'length': '99', 'type': 'image/jpeg', 'href': 'https://www.telegraph.co.uk/content/dam/news/2023/05/15/TELEMMGLPICT000335684395_trans_NvBQzQNjv4Bq7t4Eljyiy6iRMFuEKY2dXA1vLvhkMtVb21dMmpQBfEs.jpeg', 'rel': 'enclosure'}]}
{'url': 'https://www.telegraph.co.uk/football/2023/05/15/leicester-city-vs-liverpool-mnf-premier-league-live-updates/', 'title': 'Leicester City vs Liverpool live: Score and latest updates from the Premier League', 'date_published': 'Mon, 15 May 2023 17:15:48 GMT', 'image': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.telegraph.co.uk/football/2023/05/15/leicest

In [132]:
rss_results = get_rss_feed()
for article in rss_results:
    scrape_article(article)

None
no image
{'_id': '6462745151ac3347146158c7', 'url': 'https://www.telegraph.co.uk/business/2023/05/15/eu-microsoft-activision-merger-blocked-british-watchdog/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tBusiness\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': 'Brussels locked in row with Britain over $69bn Microsoft deal', 'lead': None, 'author': 'Gareth Corfield', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 5, 572941), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>Brussels is locked in a row with British regulators after waving through a $69bn (£55bn) video game takeover by Microsoft that had been blocked in the UK.</p><p>The European Commission approved Microsoft's deal to buy Call of Duty maker Activision Blizzard on Monday after concluding that competition fears were not enough to justify halting the tie-up.</p><p>It contrad

None
https://www.telegraph.co.uk/content/dam/football/2023/05/15/TELEMMGLPICT000335712709_16841714526700_trans_NvBQzQNjv4BqP4pV-m6laGcMQMbuKYgJGdHhMh81Y8GqX3ipA8YMo0k.jpeg?imwidth=680
{'_id': '6462745251ac3347146158c9', 'url': 'https://www.telegraph.co.uk/football/2023/05/15/leicester-city-vs-liverpool-mnf-premier-league-live-updates/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tSport\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': 'Leicester City vs Liverpool live: Score and latest updates from the Premier League', 'lead': None, 'author': 'Alan Tyers', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 6, 806615), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p><strong>Leicester</strong>: Iversen, Ricardo Pereira, Faes, Evans, Castagne,Tielemans, Ndidi, Soumare, Maddison, Vardy, Barnes. <strong>Subs</strong>:Smithies, Souttar, Kristiansen, A

None
https://www.telegraph.co.uk/content/dam/news/2023/05/15/TELEMMGLPICT000335687876_trans_NvBQzQNjv4BqyuLFFzXshuGqnr8zPdDWXhbQKbwj1ZsnNMhuR7O6ySI.jpeg?imwidth=480
{'_id': '6462745351ac3347146158cb', 'url': 'https://www.telegraph.co.uk/news/2023/05/15/wimborne-dorset-car-fires-war-zone-police/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tNews\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': 'Watch: Dorset town ‘like a war zone’ after 12 cars are set on fire', 'lead': None, 'author': 'Telegraph Reporters', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 7, 752045), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>Dorset residents have described a scene "like a war zone" as police investigate reports of a dozen car fires in the early hours of Monday morning.</p><p>Emergency services were called out to the scene of multiple fires in Broadstone

None
no image
{'_id': '6462745551ac3347146158ce', 'url': 'https://www.telegraph.co.uk/news/2023/05/15/monday-evening-news-briefing-sunak-and-zelensky-announce/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tNews\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': "Monday evening news briefing: Sunak and Zelensky announce 'jet coalition'", 'lead': None, 'author': 'Sam Hall', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 9, 92850), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>Good evening. Rishi Sunak met the Ukrainian president Volodymyr Zelensky at Chequers today. We have the latest, as the leaders announced they wanted to create a “jet coalition” - of which Britain is to be a “key part”.</p><h2 class="u-heading-size-large u-heading-style-normal">Sunak promises Zelensky hundreds of drones and missiles</h2><p>Britain will <a href="https://ww

None
https://www.telegraph.co.uk/content/dam/tennis/2023/05/15/TELEMMGLPICT000270960902_trans_NvBQzQNjv4BqqVzuuqpFlyLIwiB6NTmJwekapBZ1c_8650LP5YH00zA.jpeg?imwidth=680
{'_id': '6462745551ac3347146158cf', 'url': 'https://www.telegraph.co.uk/tennis/2023/05/15/andrew-richardson-coach-emma-raducanu-split-us-open/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tSport\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': "Emma Raducanu's US Open-winning coach reveals truth behind their split", 'lead': None, 'author': 'Simon Briggs', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 9, 724246), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>Andrew Richardson – the British coach who <a href="https://www.telegraph.co.uk/tennis/2021/09/11/will-win-world-reacted-emma-raducanus-us-open-final-win/">steered Emma Raducanu to the US Open title</a> – has revealed tha

...

None
no image
{'_id': '6462748551ac334714615924', 'url': 'https://www.telegraph.co.uk/personal-banking/mortgages/four-million-vast-mortgage-bill-rise-rates-peaking/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tMoney\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': 'Close to four million still face vast mortgage bill rise, despite rates peaking', 'lead': None, 'author': 'Alexa Phillips', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 57, 643026), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>Close to four million homeowners still face vast increases in their mortgage bills despite interest rates nearing their peak, a think tank has warned.</p><p><a href="https://www.telegraph.co.uk/business/2023/05/11/bank-england-interest-rate-uk-economy-growth-upgrade/">The Bank of England raised interest rates to 4.5pc this week</a> – their highest lev

None
https://www.telegraph.co.uk/content/dam/news/2023/05/12/TELEMMGLPICT000335393724_trans_NvBQzQNjv4Bq-IWLY18X4-CzgyIcjLEAj0k9u7HhRJvuo-ZLenGRumA.jpeg?imwidth=680
{'_id': '6462748651ac334714615926', 'url': 'https://www.telegraph.co.uk/news/2023/05/12/worker-sick-15-years-sues-boss-over-no-pay-rise/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tNews\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': 'IT worker on sick leave for 15 years sues over no pay rise', 'lead': None, 'author': 'Telegraph Reporters', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 58, 817554), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>A senior IT worker on sick leave since 2008 tried to sue a tech giant for discrimination because he hasn’t been given a pay rise.</p><p>Ian Clifford claimed he was the victim of disability discrimination <a href="https://www.telegrap

None
https://www.telegraph.co.uk/content/dam/news/2023/05/12/TELEMMGLPICT000335370766_trans_NvBQzQNjv4BqqVzuuqpFlyLIwiB6NTmJwfSVWeZ_vEN7c6bHu2jJnT8.jpeg?imwidth=680
{'_id': '6462748751ac334714615928', 'url': 'https://www.telegraph.co.uk/news/2023/05/12/india-demand-colonial-treasures-koh-i-noor-britain/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tNews\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': "India to force Britain into colonial 'reckoning' with treasure demands", 'lead': None, 'author': 'Craig Simpson', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 5, 59, 876952), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>India will wage a diplomatic campaign to <a href="https://www.telegraph.co.uk/royal-family/2023/05/06/queen-consort-camilla-koh-i-noor-diamond-story-history-diplomatic-jewel/">reclaim the Koh-i-Noor diamond</a> and thousands

None
https://www.telegraph.co.uk/content/dam/business/2023/05/12/TELEMMGLPICT000332608558_trans_NvBQzQNjv4Bqe8EOArUxzcOfIfa2qlTi9QxSb6CuJjZPqBRgiylT4QU.jpeg?imwidth=680
{'_id': '6462748851ac334714615929', 'url': 'https://www.telegraph.co.uk/business/2023/05/12/ftse-100-markets-live-news-uk-gdp-figures-ons/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tBusiness\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': 'UK has most stubborn inflation in developed world, Bank of America warns', 'lead': None, 'author': 'Chris Price', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 6, 0, 690523), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>Britain has the biggest problem with persistent inflation in the developed world, Bank of America's chief UK economist has said.</p><p>Robert Wood predicted the Bank of England will increase interest rates in June once

None
https://www.telegraph.co.uk/content/dam/business/2023/05/12/TELEMMGLPICT000335342548_trans_NvBQzQNjv4BqA7N2CxnJWnYI3tCbVBgu9T0aesusvN1TE7a0ddd_esI.jpeg?imwidth=680
{'_id': '6462748951ac33471461592a', 'url': 'https://www.telegraph.co.uk/business/2023/05/12/royal-mail-simon-thompson-payoff-700000-quits-unions-war/', 'primaryCategory': '\n\n\n\n\t\t\t\t\t\tBusiness\n\t\t\t\t\t\n\n\n', 'subCategories': None, 'title': 'Royal Mail boss Simon Thompson quits with £700,000 pay-off after war with unions', 'lead': None, 'author': 'Oliver Gill', 'datePublished': 'test', 'dateScraped': datetime.datetime(2023, 5, 15, 20, 6, 1, 183185), 'dateUpdated': 'test', 'language': 'en-UK', 'outlet': 'Telegraph', 'image': None, 'body': [<div class="component article-body-text" data-test="article-body-text">
<p>Simon Thompson is set for a pay-off of up to £700,000 after quitting as the boss of Royal Mail in the wake of a <a href="https://www.telegraph.co.uk/business/2023/05/10/royal-mail-trade-unions-pay-di

In [118]:
for article in rss_results:
    print(article['image'][0]['url'])

https://www.telegraph.co.uk/news/2023/05/15/mountain-rescue-dog-scafell-pike-lake-district/
https://www.telegraph.co.uk/football/2023/05/15/leicester-city-vs-liverpool-mnf-premier-league-live-updates/
https://www.telegraph.co.uk/music/artists/beyonce-renaissance-tour-team-whos-who/
https://www.telegraph.co.uk/news/2023/05/15/wimborne-dorset-car-fires-war-zone-police/
https://www.telegraph.co.uk/tax/news/extra-240000-taxpayers-fines-under-latest-capital-gain-raid/
https://www.telegraph.co.uk/royal-family/2023/05/15/prince-harry-phone-hacking-high-court-trial-omid-scobie/
https://www.telegraph.co.uk/news/2023/05/15/monday-evening-news-briefing-sunak-and-zelensky-announce/
https://www.telegraph.co.uk/tennis/2023/05/15/andrew-richardson-coach-emma-raducanu-split-us-open/
https://www.telegraph.co.uk/news/2023/05/15/russias-flanks-collapse-in-bakhmut-due-to-poorly-trained/
https://www.telegraph.co.uk/travel/destinations/oceania/australia/new-south-wales/sydney/articles/best-luxury-hotels-syd